In [25]:
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [26]:
import gensim

model = gensim.models.Word2Vec.load("latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [27]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [28]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [29]:
quesstions_list = worksheet.col_values(2)#B
quesstions_list = quesstions_list[1:]

answers_list = worksheet.col_values(3)#C
answers_list = answers_list[1:]

detail_list = worksheet.col_values(4)#D
detail_list = detail_list[1:]

In [30]:
import MeCab

# 取り出したい品詞
select_conditions = [ '固有名詞']

# 分かち書きオブジェクト
tagger = MeCab.Tagger('')

tagger.parse('')

def wakati_text(text):

    # 分けてノードごとにする
    node = tagger.parseToNode(text)
    terms = []

    while node:

        # 単語
        term = node.surface

        # 品詞
        pos = node.feature.split(',')[1]

        # もし品詞が条件と一致してたら
        if pos in select_conditions:
            terms.append(term)

        node = node.next

    # 連結おじさん
    text_result = ' '.join(terms)
    return text_result

In [33]:

for answer,quesstion, text in zip(answers_list, quesstions_list, detail_list):
    
    print(answer + " " +wakati_text(answer)  + " " + wakati_text(quesstion) + " " + wakati_text(text))
    
    try:
        a = model.wv[answer]
    except:
        print(answer)